In [1]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast

tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
model = DPRContextEncoder.from_pretrained("../temp/dpr/models/ctx_encoder_dpr")

/opt/conda/envs/read/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [2]:
import pandas as pd

df = pd.read_json("../data/totto2/filtered/train.jsonl", lines=True)

In [4]:
import json

df["table_values"] = df["table"].apply(lambda x: pd.DataFrame(json.loads(x)).values.flatten().tolist())

In [5]:
import itertools


all_values = list(itertools.chain.from_iterable(df["table_values"].tolist()))

In [8]:
import random

sampled_values = random.sample(all_values, 100)

In [10]:
labels = []
import re
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load('ner')


for value in sampled_values:
    if not re.search('[A-Za-z]', value):
        labels.append("NUM")
    else:
        sentence = Sentence(value)
        tagger.predict(sentence)
        for entity in sentence.get_spans('ner'):
            tag = entity.get_label("ner").value
            labels.append("ENTITY")
            break
        else:
            labels.append("TEXT")

2022-09-22 17:33:45,075 https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmphhkznqfi


100%|██████████| 432197603/432197603 [01:06<00:00, 6451534.95B/s] 

2022-09-22 17:34:52,673 copying /tmp/tmphhkznqfi to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2022-09-22 17:34:52,938 removing temp file /tmp/tmphhkznqfi
2022-09-22 17:34:52,975 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


AttributeError: 'KeyedVectors' object has no attribute 'key_to_index'